This tutorial explains details about existing parsers and how to create a new parser if necessary.

# 2021 Google Android Derived Dataset

This data comes from the 2021 Google Smartphone Decimeter Challenge and can be downloaded from [Kaggle](https://www.kaggle.com/c/google-smartphone-decimeter-challenge).

In [ ]:
import os

from gnss_lib_py.parsers.android import AndroidDerived2021
from gnss_lib_py.utils.file_operations import get_lib_dir

Loading the data into an instance of `NavData` is as easy as creating an instance of `AndroidDerived2021` with the relevant file path.

In [ ]:
# download Android data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
# load Android Google Challenge data
derived_data = AndroidDerived2021("Pixel4XL_derived.csv")

We can verify that the data loaded correctly by printing the shape and rows of the imported data.

In [ ]:
derived_data.shape

In [ ]:
derived_data.rows

# How to Create a New NavData Class

The modular and versatile functionality of this gnss_lib_py repository is enabled by loading all data types into a custom Python NavData class. If you are using a type of data or dataset that is not yet supported, you will need to create a new child class of the NavData Python class. This tutorial will guide you on how to set up your new Python child class. Once complete, please feel free to submit a pull request to our GitHub repository so other users can  make use of the added functionality.

For this example, say that we have a new type of data called MyReceiver that is a csv file with columns of a timestamp, satellite identifier, and pseudorange. The contents of a sample `myreceiver.csv` is the following:

| myTimestamp | mySatId | myPseudorange |
| ----------- | ------- | ------------- |
| 10          | G10     | 270000001     |
| 10          | G14     | 270000007     |
| 10          | G07     | 270000004     |
| 10          | G03     | 270000005     |
| 11          | G10     | 270000002     |
| 11          | G14     | 270000008     |
| 11          | G07     | 270000003     |
| 11          | G03     | 270000004     |

The first step is importing the base `NavData` class and creating a new class type that inherets from `NavData`

```python
class MyReceiver(NavData):
    """Class handling measurements from MyReceiver.

    Inherits from NavData().
    """
```

The `__init__` function should have a call to the parent `NavData` `__init__` function. Based on your data input, you should call the corresponding initializer.

For csv files, call: `super().__init__(csv_path=input_path)`  
For pandas DataFrames, call `super().__init__(pandas_df=input_path)`  
For numpy ndarrays, call `super().__init__(numpy_array=input_path)`  

In our case, we have a csv file, so our `__init__` function looks like the following:

```python
def __init__(self, input_path):
    """MyReceive specific loading and preprocessing

    Parameters
    ----------
    input_path : string
        Path to MyReceiver csv file

    """

    # call NavData initialization with csv path
    super().__init__(csv_path=input_path)
```

After our data is loaded, we may want to make known changes to our data. We can make those changes by defining a `postprocess` function. NavData's `__init__` function that we call in our initializer already makes a call to the `postprocess` function, so we don't have to repeat that call in `MyReceiver`'s `__init__` function.

One thing that we need to do to make use of the common functionality of `gnss_lib_py` is to standardize the names of our variables. See the [Standard Naming Conventions](https://gnss-lib-py.readthedocs.io/en/latest/reference/reference.html#standard-naming-conventions) section in the Reference tab of the documentation for the list of standardized names.

In our case, we will convert `mySatId` to `sv_id` and `myPseudorange` to `raw_pr_m`. We make these conversions by simply updating the `_row_map` function.

```python


@staticmethod
def _row_map():
    """Map of column names from loaded to gnss_lib_py standard

    Returns
    -------
    row_map : Dict
        Dictionary of the form {old_name : new_name}
    """
    row_map = {'mySatId' : 'sv_id',
               'myPseudorange' : 'raw_pr_m',
               }
    return row_map
```

As an additional postprocessing step, we may want to offset our pseudorange due to a known error or create the common timestamp variable based on our unique timestamp row.

```python
# correct pseudorange
self['corr_pr_m'] = self['raw_pr_m'] + 100.

# create common timestamp
self['unix_time_millis'] = self['myTimestamp'] + 1659075505350
```

Putting it all together, we have:

In [ ]:
from gnss_lib_py.parsers.navdata import NavData

class MyReceiver(NavData):
    """Class handling measurements from MyReceiver.

    Inherits from NavData().
    """
    def __init__(self, input_path):
        """MyReceive specific loading and preprocessing

        Parameters
        ----------
        input_path : string
            Path to MyReceiver csv file

        """
        
        # call NavData initialization with csv path
        super().__init__(csv_path=input_path)
        
    def postprocess(self):
        """MyReceiver specific postprocessing

        """

        # correct pseudorange
        self['corr_pr_m'] = self['raw_pr_m'] + 100.
        
        # create common timestamp
        self['unix_time_millis'] = self['myTimestamp'] + 1659075505350
        

    @staticmethod
    def _row_map():
        """Map of column names from loaded to gnss_lib_py standard

        Returns
        -------
        row_map : Dict
            Dictionary of the form {old_name : new_name}
        """
        row_map = {'mySatId' : 'sv_id',
                   'myPseudorange' : 'raw_pr_m',
                   }
        return row_map

We can now create a instance of our new `MyReceiver` class with the path to our csv called `myreceiver.csv`.

In [ ]:
# download myreceiver.csv file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/notebooks/tutorials/myreceiver.csv --quiet -O "myreceiver.csv"

# create instance of MyReceiver
my_receiver_data = MyReceiver("myreceiver.csv")

Let's print out our corrected pseudorange to make sure everything worked correctly.

In [ ]:
my_receiver_data["corr_pr_m"]

We can now take advantage of all the tools `gnss_lib_py` has to offer!

In [ ]:
from gnss_lib_py.utils.visualizations import plot_metric
fig = plot_metric(my_receiver_data,"myTimestamp","corr_pr_m",save=False)